# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 16 2022 11:46AM,244265,10,0085939081,ISDIN Corporation,Released
1,Aug 16 2022 11:46AM,244265,10,0085939104,ISDIN Corporation,Released
2,Aug 16 2022 11:46AM,244265,10,0085939109,ISDIN Corporation,Released
3,Aug 16 2022 11:46AM,244265,10,0085939200,ISDIN Corporation,Released
4,Aug 16 2022 11:46AM,244265,10,0085939213,ISDIN Corporation,Released
5,Aug 16 2022 11:46AM,244265,10,0085939214,ISDIN Corporation,Released
6,Aug 16 2022 11:45AM,244264,10,Enova-10491,Emerginnova,Released
7,Aug 16 2022 11:45AM,244264,10,DNMD-23638,Emerginnova,Released
8,Aug 16 2022 11:44AM,244263,10,8409911,"Snap Medical Industries, LLC",Released
9,Aug 16 2022 11:44AM,244263,10,8409912,"Snap Medical Industries, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
37,244261,Released,1
38,244262,Released,1
39,244263,Released,2
40,244264,Released,2
41,244265,Released,6


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
244261,NaN,NaN,1.0
244262,NaN,NaN,1.0
244263,NaN,NaN,2.0
244264,NaN,NaN,2.0
244265,NaN,NaN,6.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244164,0.0,0.0,1.0
244183,0.0,0.0,1.0
244189,0.0,0.0,1.0
244191,0.0,0.0,1.0
244192,0.0,0.0,12.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244164,0,0,1
244183,0,0,1
244189,0,0,1
244191,0,0,1
244192,0,0,12


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244164,0,0,1
1,244183,0,0,1
2,244189,0,0,1
3,244191,0,0,1
4,244192,0,0,12


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244164,,,1
1,244183,,,1
2,244189,,,1
3,244191,,,1
4,244192,,,12


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 16 2022 11:46AM,244265,10,ISDIN Corporation
6,Aug 16 2022 11:45AM,244264,10,Emerginnova
8,Aug 16 2022 11:44AM,244263,10,"Snap Medical Industries, LLC"
10,Aug 16 2022 11:42AM,244262,19,ACG North America LLC
11,Aug 16 2022 11:42AM,244261,16,Sartorius Bioprocess Solutions
12,Aug 16 2022 11:41AM,244259,20,Alumier Labs Inc.
24,Aug 16 2022 11:38AM,244260,19,"GUSA Granules USA, Inc."
26,Aug 16 2022 11:38AM,244245,20,Alumier Labs Inc.
47,Aug 16 2022 11:30AM,244258,20,"Andelyn Biosciences, Inc."
48,Aug 16 2022 11:28AM,244257,19,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 16 2022 11:46AM,244265,10,ISDIN Corporation,,,6
1,Aug 16 2022 11:45AM,244264,10,Emerginnova,,,2
2,Aug 16 2022 11:44AM,244263,10,"Snap Medical Industries, LLC",,,2
3,Aug 16 2022 11:42AM,244262,19,ACG North America LLC,,,1
4,Aug 16 2022 11:42AM,244261,16,Sartorius Bioprocess Solutions,,,1
5,Aug 16 2022 11:41AM,244259,20,Alumier Labs Inc.,,,12
6,Aug 16 2022 11:38AM,244260,19,"GUSA Granules USA, Inc.",,,2
7,Aug 16 2022 11:38AM,244245,20,Alumier Labs Inc.,,,21
8,Aug 16 2022 11:30AM,244258,20,"Andelyn Biosciences, Inc.",,,1
9,Aug 16 2022 11:28AM,244257,19,"AdvaGen Pharma, Ltd",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 16 2022 11:46AM,244265,10,ISDIN Corporation,6,,
1,Aug 16 2022 11:45AM,244264,10,Emerginnova,2,,
2,Aug 16 2022 11:44AM,244263,10,"Snap Medical Industries, LLC",2,,
3,Aug 16 2022 11:42AM,244262,19,ACG North America LLC,1,,
4,Aug 16 2022 11:42AM,244261,16,Sartorius Bioprocess Solutions,1,,
5,Aug 16 2022 11:41AM,244259,20,Alumier Labs Inc.,12,,
6,Aug 16 2022 11:38AM,244260,19,"GUSA Granules USA, Inc.",2,,
7,Aug 16 2022 11:38AM,244245,20,Alumier Labs Inc.,21,,
8,Aug 16 2022 11:30AM,244258,20,"Andelyn Biosciences, Inc.",1,,
9,Aug 16 2022 11:28AM,244257,19,"AdvaGen Pharma, Ltd",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 16 2022 11:46AM,244265,10,ISDIN Corporation,6,,
1,Aug 16 2022 11:45AM,244264,10,Emerginnova,2,,
2,Aug 16 2022 11:44AM,244263,10,"Snap Medical Industries, LLC",2,,
3,Aug 16 2022 11:42AM,244262,19,ACG North America LLC,1,,
4,Aug 16 2022 11:42AM,244261,16,Sartorius Bioprocess Solutions,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 16 2022 11:46AM,244265,10,ISDIN Corporation,6.0,NaN,NaN
1,Aug 16 2022 11:45AM,244264,10,Emerginnova,2.0,NaN,NaN
2,Aug 16 2022 11:44AM,244263,10,"Snap Medical Industries, LLC",2.0,NaN,NaN
3,Aug 16 2022 11:42AM,244262,19,ACG North America LLC,1.0,NaN,NaN
4,Aug 16 2022 11:42AM,244261,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 16 2022 11:46AM,244265,10,ISDIN Corporation,6.0,0.0,0.0
1,Aug 16 2022 11:45AM,244264,10,Emerginnova,2.0,0.0,0.0
2,Aug 16 2022 11:44AM,244263,10,"Snap Medical Industries, LLC",2.0,0.0,0.0
3,Aug 16 2022 11:42AM,244262,19,ACG North America LLC,1.0,0.0,0.0
4,Aug 16 2022 11:42AM,244261,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3419394,119.0,4.0,0.0
15,732653,67.0,55.0,2.0
16,1221058,4.0,1.0,0.0
19,1221149,5.0,1.0,0.0
20,1465427,61.0,0.0,0.0
21,732583,3.0,0.0,0.0
22,488506,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3419394,119.0,4.0,0.0
1,15,732653,67.0,55.0,2.0
2,16,1221058,4.0,1.0,0.0
3,19,1221149,5.0,1.0,0.0
4,20,1465427,61.0,0.0,0.0
5,21,732583,3.0,0.0,0.0
6,22,488506,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,119.0,4.0,0.0
1,15,67.0,55.0,2.0
2,16,4.0,1.0,0.0
3,19,5.0,1.0,0.0
4,20,61.0,0.0,0.0
5,21,3.0,0.0,0.0
6,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,119.0
1,15,Released,67.0
2,16,Released,4.0
3,19,Released,5.0
4,20,Released,61.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21,22
Status,,,,,,,
Completed,0.0,2.0,0.0,0.0,0.0,0.0,0.0
Executing,4.0,55.0,1.0,1.0,0.0,0.0,0.0
Released,119.0,67.0,4.0,5.0,61.0,3.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21,22
0,Completed,0.0,2.0,0.0,0.0,0.0,0.0,0.0
1,Executing,4.0,55.0,1.0,1.0,0.0,0.0,0.0
2,Released,119.0,67.0,4.0,5.0,61.0,3.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21,22
0,Completed,0.0,2.0,0.0,0.0,0.0,0.0,0.0
1,Executing,4.0,55.0,1.0,1.0,0.0,0.0,0.0
2,Released,119.0,67.0,4.0,5.0,61.0,3.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()